# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import azureml
import joblib
import os

from azureml.core import Environment
from azureml.core import ScriptRunConfig
from azureml.core.runconfig import DockerConfiguration
from azureml.core.dataset import Dataset


from azureml.core import Workspace, Experiment

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from azureml.core.runconfig import EnvironmentDefinition, CondaDependencies

from azureml.train.hyperdrive import (
    BayesianParameterSampling,
    HyperDriveConfig,
    quniform,
    uniform,
    choice,
    PrimaryMetricGoal,
)

from azureml.widgets import RunDetails

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.42.0


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
experiment_name = 'hyperdrive-lgbm'

experiment=Experiment(ws, experiment_name)
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: quick-starts-ws-200268
Azure region: southcentralus
Subscription id: 9a7511b8-150f-4a58-8528-3e7d50216c31
Resource group: aml-quickstarts-200268


In [3]:
# Create or attach a compute cluster

cluster_name = "canina-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('This compute target already exists.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2",
                                                          max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
    print('A new compute target has been created.')
    
compute_target.wait_for_completion(show_output=True, min_node_count=0, timeout_in_minutes=10)

This compute target already exists.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
train_dataset = Dataset.get_by_name(ws, name='train')
train_id = train_dataset.id

In [5]:
auxi_dataset = Dataset.get_by_name(ws, name='auxi')
auxi_id = auxi_dataset.id

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [6]:
# Configure Docker environment 
env = EnvironmentDefinition()
env.python.user_managed_dependencies = False
env.python.conda_dependencies = CondaDependencies.create(
    conda_packages=["pandas", "numpy", "scipy", "scikit-learn", "lightgbm", "joblib"],
    python_version="3.6.2"
)
env.python.conda_dependencies.add_channel("conda-forge")

docker_config = DockerConfiguration(use_docker=True)

In [7]:
# Training script
script_folder = "./hyperdrive"
train_script_name = "train.py"

src = ScriptRunConfig(source_directory=script_folder,
                      script=train_script_name,
                      arguments=["--train", train_id, "--auxi", auxi_id],
                      compute_target=compute_target,
                      environment=env,
                      docker_runtime_config=docker_config)

# Create the different params used during training
param_sampling = BayesianParameterSampling(
    {
        "--num-leaves": quniform(8, 128, 1),
        "--min-data-in-leaf": quniform(20, 500, 10),
        "--learning-rate": choice(
            1e-4, 1e-3, 5e-3, 1e-2, 1.5e-2, 2e-2, 3e-2, 5e-2, 1e-1
        ),
        "--feature-fraction": uniform(0.2, 1),
        "--bagging-fraction": uniform(0.1, 1),
        "--bagging-freq": quniform(1, 20, 1),
        "--max-rounds": quniform(50, 2000, 10),
        "--max-lag": quniform(3, 40, 1),
        "--window-size": quniform(3, 40, 1)
    }
)

hyperdrive_run_config = HyperDriveConfig(
    run_config=src,
    hyperparameter_sampling=param_sampling,
    primary_metric_name="MAPE",
    primary_metric_goal=PrimaryMetricGoal.MINIMIZE,
    max_total_runs=25,
    max_concurrent_runs=4
)

For best results with Bayesian Sampling we recommend using a maximum number of runs greater than or equal to 20 times the number of hyperparameters being tuned. Recommendend value:180.


In [8]:
# Submit the experiment
hyperdrive_run = experiment.submit(config=hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [10]:
# We need to wait before retrieving the best model
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_f7ece0dc-64e5-4666-8f2e-2223c5cd790e
Web View: https://ml.azure.com/runs/HD_f7ece0dc-64e5-4666-8f2e-2223c5cd790e?wsid=/subscriptions/9a7511b8-150f-4a58-8528-3e7d50216c31/resourcegroups/aml-quickstarts-200268/workspaces/quick-starts-ws-200268&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Execution Summary
RunId: HD_f7ece0dc-64e5-4666-8f2e-2223c5cd790e
Web View: https://ml.azure.com/runs/HD_f7ece0dc-64e5-4666-8f2e-2223c5cd790e?wsid=/subscriptions/9a7511b8-150f-4a58-8528-3e7d50216c31/resourcegroups/aml-quickstarts-200268/workspaces/quick-starts-ws-200268&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'HD_f7ece0dc-64e5-4666-8f2e-2223c5cd790e',
 'target': 'canina-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-07-05T16:24:11.675399Z',
 'endTimeUtc': '2022-07-05T17:03:30.912222Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "MAPE", "goal": "minimize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'dfaad5d7-59e0-4ede-aaf2-326398e2f32b',
  'user_agent': 'python/3.8.5 (Linux-5.4.0-1083-azure-x86_64-with-glibc2.10) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.42.0',
  'space_size': 'infinite_space_size',
  'score': '30.151345591841505',
  'best_child_run_id': 'HD_f7ece0dc-64e5-4666-8f2e-2223c5cd790e_23',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_f7ece0dc-64e5-4666-8f2e-2223c5cd790e_23'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg200268.blob.core.

In [11]:
# View the different models trained and their performance
hyperdrive_run.get_metrics()

{'HD_f7ece0dc-64e5-4666-8f2e-2223c5cd790e_23': {'MAPE': 30.151345591841505},
 'HD_f7ece0dc-64e5-4666-8f2e-2223c5cd790e_22': {'MAPE': 73.25484753553937},
 'HD_f7ece0dc-64e5-4666-8f2e-2223c5cd790e_24': {'MAPE': 31.745190714812683},
 'HD_f7ece0dc-64e5-4666-8f2e-2223c5cd790e_20': {'MAPE': 33.717307153649784},
 'HD_f7ece0dc-64e5-4666-8f2e-2223c5cd790e_21': {'MAPE': 56.70119392418192},
 'HD_f7ece0dc-64e5-4666-8f2e-2223c5cd790e_19': {'MAPE': 40.558109506551624},
 'HD_f7ece0dc-64e5-4666-8f2e-2223c5cd790e_18': {'MAPE': 36.446836912017694},
 'HD_f7ece0dc-64e5-4666-8f2e-2223c5cd790e_17': {'MAPE': 37.55781117184272},
 'HD_f7ece0dc-64e5-4666-8f2e-2223c5cd790e_16': {'MAPE': 47.73128629174131},
 'HD_f7ece0dc-64e5-4666-8f2e-2223c5cd790e_12': {'MAPE': 47.88451746273107},
 'HD_f7ece0dc-64e5-4666-8f2e-2223c5cd790e_15': {'MAPE': 70.36378573559841},
 'HD_f7ece0dc-64e5-4666-8f2e-2223c5cd790e_14': {'MAPE': 51.40861303411739},
 'HD_f7ece0dc-64e5-4666-8f2e-2223c5cd790e_13': {'MAPE': 63.85008697676034},
 'HD_f7

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [12]:
best_run_hyperdrive = hyperdrive_run.get_best_run_by_primary_metric()

# Print the best model's Run Id and metrics
best_metrics = best_run_hyperdrive.get_metrics()
print('Run Id : ', best_run_hyperdrive.id)
print('Metrics :',best_metrics)

Run Id :  HD_f7ece0dc-64e5-4666-8f2e-2223c5cd790e_23
Metrics : {'MAPE': 30.151345591841505}


In [13]:
# Get the best hyperparameter values
best_run_hyperdrive.get_details()['runDefinition']['arguments']

['--train',
 '68928728-fbcc-4616-8f45-b8bd4d505143',
 '--auxi',
 '153b08ae-28c2-4ffe-bc03-16c2e57d40b4',
 '--num-leaves',
 '113',
 '--min-data-in-leaf',
 '370',
 '--learning-rate',
 '0.03',
 '--feature-fraction',
 '0.7625931347901467',
 '--bagging-fraction',
 '0.46279264150238275',
 '--bagging-freq',
 '16',
 '--max-rounds',
 '1640',
 '--max-lag',
 '4',
 '--window-size',
 '30']

In [14]:
# Display the saved file of the best model
best_run_hyperdrive.get_file_names()

['logs/azureml/dataprep/0/backgroundProcess.log',
 'logs/azureml/dataprep/0/backgroundProcess_Telemetry.log',
 'logs/azureml/dataprep/0/rslex.log.2022-07-05-16',
 'outputs/model.joblib',
 'system_logs/cs_capability/cs-capability.log',
 'system_logs/hosttools_capability/hosttools-capability.log',
 'system_logs/lifecycler/execution-wrapper.log',
 'system_logs/lifecycler/lifecycler.log',
 'system_logs/lifecycler/vm-bootstrapper.log',
 'system_logs/metrics_capability/metrics-capability.log',
 'user_logs/std_log.txt']

In [15]:
# Download the best model
best_run_hyperdrive.download_file('outputs/model.joblib',
                                 output_file_path = './outputs/')

In [16]:
joblib.load('./outputs/model.joblib')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [17]:
# Register the best model
best_hyperdrive_model = best_run_hyperdrive.register_model(model_name = 'hyperdrive-lgbm', model_path = 'outputs/model.joblib')

In [18]:
best_hyperdrive_model

Model(workspace=Workspace.create(name='quick-starts-ws-200268', subscription_id='9a7511b8-150f-4a58-8528-3e7d50216c31', resource_group='aml-quickstarts-200268'), name=hyperdrive-lgbm, id=hyperdrive-lgbm:1, version=1, tags={}, properties={})

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

